In [27]:
from pathlib import Path

import numpy as np

In [29]:
import skimage.transform


def read_env(filename):
    hdr_info = ("id", "frames", "lines", "samples", "samplesize")
    hdr, timestamps, data = {}, None, None
    with open(filename, "rb") as raw_bytes:
        # read 4 bytes header
        for info in hdr_info:
            hdr[info] = int.from_bytes(raw_bytes.read(4), byteorder="little")
        # read timestamps and data
        timestamps = np.zeros(hdr["frames"], dtype="int64")
        sz = hdr["lines"] * hdr["samples"] * hdr["samplesize"]
        data = np.zeros((hdr["lines"], hdr["samples"], hdr["frames"]), dtype="uint8")
        for frame in range(hdr["frames"]):
            # read 8 bytes of timestamp
            timestamps[frame] = int.from_bytes(raw_bytes.read(8), byteorder="little")
            # read each frame
            data[:, :, frame] = np.frombuffer(
                raw_bytes.read(sz), dtype="uint8"
            ).reshape([hdr["lines"], hdr["samples"]])
    print(
        "Loaded {d[2]} raw frames of size, {d[0]} x {d[1]} (lines x samples)".format(
            d=data.shape
        )
    )
    target_shape = (2928, 192)
    dataEnv = np.transpose(data, [1, 0, 2])
    data = np.zeros([2928, 192, dataEnv.shape[2]])
    for ii in range(0, dataEnv.shape[2]):
        data[:, :, ii] = skimage.transform.resize(
            dataEnv[:, :, ii], target_shape, anti_aliasing=True
        )
    data = np.flip(data, axis=1)
    return hdr, timestamps, data

In [41]:
file_path = Path("/Volumes/CREST Data/David_S_Data/Clarius_RF_Data/a896ef50-1922-4df2-8995-a57b5a46d800/2024-10-29T13-00-21+0000_rf.raw.lzo")
read_env(file_path)

ValueError: array is too big; `arr.size * arr.dtype.itemsize` is larger than the maximum possible size.

In [42]:
hdr_info = ("id", "frames", "lines", "samples", "samplesize")
hdr, timestamps, data = {}, None, None
with open(file_path, "rb") as f:
    for info in hdr_info:
        hdr[info] = int.from_bytes(f.read(4), byteorder="little")

In [43]:
hdr

{'id': 1331317897,
 'frames': 436866304,
 'lines': 537923594,
 'samples': 20973872,
 'samplesize': 773}